 # Aprendizaje Automático : Trabajo Práctico 2


# Integrantes: Juan Manuel Perez, Mariela Rajngewerc, Tomás Freilij

# Resumen:

Para el presente trabajo implementamos el algoritmo Q-Learning para la resolucion del juego Cuatro en linea. El objetivo será mostrar cómo un jugador guiado por el algoritmo en cuestión, termina superando ampliamente a uno que elija todos sus movimientos al azar.


# Palabras clave: 

aprendizaje por refuerzos, Q-Learning, cuatro en linea, 

# Desarrollo

El trabajo consistió en armar una implementación del cuatro en línea, que nos permitiera simular fielmente el desarrollo de una partida. Para eso primero lo armamos para que puedan jugar dos humanos.

En una segunda etapa, agregamos el jugador random (lo cual no presentó muchas dificultades) y un Qplayer, que va a jugar guiado por el algoritmo en cuestión.

Nuestra hipotesis es que cuanto mas entrenamiento realicemos sobre el QPLayer, la tabla Q sera actualizada de forma mas eficiente y asi, dado un estado y una accion el QPlayer sabra cuan conveniente es tomar esa accion dado el estado que se encuentra. 

Al momento de elegir una accion dado un estado decidimos no usar siempre la experiencia que va adquiriendo el QPlayer sino, con probabilidad epsilon, elegir alguna accion posible al azar. La idea es que el jugador no juegue solo en con las acciones conocidas y que logre experimentar nuevas acciones.

Por la escencia del algoritmo por refuerzos elegido: Q-Learning, debimos considerar y experimentar valores para los siguientes hiperparametros: alpha y gamma.





# Estrategia 

Como en el ajedrez, en este juego existe la posibilidad de, antes de hacer un movimiento, explorar las posibles respuestas al mismo, para así tener un mejor análisis de qué jugar, o de en qué situación estamos (si perdiendo, ganando o empatando).

Por ejemplo, si estamos en un estado del juego en el que una jugada nos lleva a perder de forma obligada, es evidente que no optaríamos por la misma. A su vez, si una nos lleva a ganar de forma directa, tenemos que ir por ese camino.

Es de esta forma, que nuestro Qplayer, haciendo este tipo de análisis, irá aprendiendo. En este caso, aprender es equivalente a aumentar la probabilidad de ganar en una futura partida.





A. Descripcion y uso de la implementacion

B. Calculo de las funciones: Recompensa y Q
La funcion recompensa le adjudica a cada (estado, accion) un valor real. Representa el peso que tiene la casilla donde el QPlayer pondra ficha (osea, la accion que ejercera). Decidimos considerar los siguientes resultados posibles:
-recompensa_max: en el caso de que el QPlayer resulte ganados
-recompensa_media: en el caso que ya hubiera 2 fichas concecutivas del Qplayer, se pueda agregar una tercera y haya un lugar libre donde, en jugadas posteriores podria completarse la cuarta ficha.
-recompensa_baja:en el caso que ya hubiera 2 fichas concecutivas del jugador rival, se pueda agregar una tercera y haya un lugar libre donde, en jugadas posteriores podria completarse la cuarta ficha. 
-recompensa_cero=0


-casos que no considere: 1010 por ejemplo, seria bueno poner donde hay cero-
C. Calculo 

Resultados

Conclusiones

# 1) Buscar alpha y gamma, hacer el cuadro. Con esto se puede intuir a priori como influye la tasa de aprendizaje.

Experimento realizado para epsilon=0,2

# 2) Una vez que conseguimos la mejor combinacion, tiro distintos juegos entrenamientos a ver el porcentaje de ganancias
25
50
75
100
150
200
250
etc

Experimento realizado para epsilon=0,2

# 3) Pruebo lo mismo que en 2 pero con distintas inicializaciones para Q, podria ser un random entre los distintos rewards y por ahi uno aleatorio de una.


# 4) Cambiar la exploracion por la funcion de proba SOFTMAX



Algunos resultados:
Diccionarios {alpha, gamma: cantidad de ganadas por Qplayer}
1)  epsilon 0.2
    alphas = [0,0.25,0.5,0.75,1]
    gammas = [0,0.25,0.5,0.75,1]
    Tirando 100 corridas, obtuvimos
    {(0, 0.5): 79, (0.5, 1): 78, (0.75, 0): 72, (0.25, 0.25): 78, (1, 0.5): 85, (0.5, 0.5): 70, (0.75, 0.75): 81, (1, 0.25): 78, (0.75, 0.5): 82, (1, 1): 70, (0.25, 1): 75, (0.25, 0): 76, (0.5, 0): 77, (0, 0.25): 76, (0, 0.75): 82, (0.25, 0.75): 76, (1, 0): 73, (0.5, 0.25): 69, (1, 0.75): 76, (0, 1): 82, (0, 0): 78, (0.75, 1): 78, (0.5, 0.75): 73, (0.75, 0.25): 80, (0.25, 0.5): 82}

2)  epsilon 0.2
    alphas = [0,1]
    gammas = [0,1]
   Tirando 500 corridas, obtuvimos:
{(0, 1): 387, (1, 0): 391, (0, 0): 364, (1, 1): 393}
3)  epsilon 0.2
    alphas = 
    gammas = 
    Tirando 200 corridas obtuvimos:
    {(0.25, 1): 155, (0.25, 0.5): 157, (0.75, 0): 144, (0.75, 0.75): 158, (0.5, 0.75): 150, (0.5, 0.5): 154, (0.5, 0): 158, (0.75, 0.25): 155, (0.25, 0.25): 150, (0.25, 0): 145, (0.75, 0.5): 150, (0.25, 0.75): 150, (1, 0.75): 146, (1, 0.5): 163, (0.5, 0.25): 153, (1, 0): 157, (1, 0.25): 155}
    
    
    FALTAN LOS CASOS (0.5,0) (0.25,0.75) (0.25, 1) (0.75,1) (1,1)
    
    
    
   # Empiezo a experimentar con alpha=1 y gamma=0.5 ya que en ambos experimentos fue superadora
   QPLAYER VS RANDOM PLAYER
   -10 PARTIDAS: {(1, 0.5): 8};{(1, 0.5): 7};{(1, 0.5): 6};{(1, 0.5): 7}
   -20 PARTIDAS: {(1, 0.5): 14}
   -30 PARTIDAS: {(1, 0.5): 23}
   -50 PARTIDAS: {(1, 0.5): 38}
   -99 PARTIDAS: {(1, 0.5): 69}
   -100 PARTIDAS: {(1, 0.5): 79}; {(1, 0.5): 74};{(1, 0.5): 73};{(1, 0.5): 77};{(1, 0.5): 83}:{(1, 0.5): 72}
   -200 PARTIDAS: {(1, 0.5): 149}
   -250 PARTIDAS: {(1, 0.5): 197}
   -300 PARTIDAS:{(1, 0.5): 232}
   -400 PARTIDAS: {(1, 0.5): 303}
   -500 partidas: {(1, 0.5): 363};{(1, 0.5): 387}
   -1000 partidas: {(1, 0.5): 760}
   
   